In [129]:
# To start mysql docker: sudo docker-compose up -d

import datajoint as dj
import os
import sys
import json
import re
import warnings
from scipy.io import loadmat

In [130]:
class CorruptData(Exception):
    pass

class InconsistentData(Warning):
    pass

In [2]:
root_path = os.path.abspath(os.path.join(globals()['_dh'][0], '../..'))

In [3]:
local_cred_filename = os.path.join(root_path, 'datajoint/local_cred.ini')
with open(local_cred_filename) as f:
    local_cred = json.load(f)

In [4]:
local_cred

In [5]:
cred_filename = "~/"

dj.config['database.host'] = local_cred['host']
dj.config['database.user'] = local_cred['user']
dj.config['database.password'] = local_cred['password']

In [6]:
schema = dj.schema('franklab_nspike', locals())

In [201]:
schema.drop()

In [165]:
anim.drop()

In [166]:

@schema
class Animal(dj.Manual):
    definition = """
    anim_name: varchar(20)  #Name of animal
    ---
    anim_path_raw: varchar(200)
    anim_path_mat: varchar(200)
    """

@schema
class Day(dj.Imported):
    definition = """
    -> Animal
    day: int
    ---
    day_path_raw: varchar(200)
    day_start_time_sec: float
    day_end_time_sec: float
    day_start_time_nspike: int
    day_end_time_nspike: int
    """
    
    def make(self, key):
        
        anim_name, anim_path_raw, anim_path_mat = (Animal() & key).fetch1('anim_name', 'anim_path_raw', 'anim_path_mat')
        dir_names = os.listdir(anim_path_raw)
        for dir_name in dir_names:
            m = re.search('^{:s}(\d*)$'.format(anim_name.lower()), dir_name)
            if m:
                day = int(m.groups()[0])
                day_path_raw = os.path.join(anim_path_raw, dir_name)
                times_path = os.path.join(day_path_raw, 'times.mat')
                if os.path.isfile(times_path):
                    times_mat = loadmat(times_path)
                    time_ranges = times_mat['ranges']
                    day_start_time_nspike = time_ranges[0][0]
                    day_end_time_nspike = time_ranges[0][1]
                    day_start_time_sec = day_start_time_nspike/10000
                    day_end_time_sec = day_end_time_nspike/10000
                    self.insert1({'anim_name': anim_name,
                                  'day': day,
                                  'day_path_raw': day_path_raw,
                                  'day_start_time_sec': day_start_time_nspike,
                                  'day_end_time_sec': day_end_time_nspike,
                                  'day_start_time_nspike': day_start_time_sec,
                                  'day_end_time_nspike': day_end_time_sec})
                else:
                    # Missing times.mat means data folder was not processed for spike sorting (matclust)
                    pass

@schema
class Epoch(dj.Imported):
    definition = """
    -> Day
    epoch_id: tinyint
    ---
    epoch_name: varchar(50)
    epoch_time_str: varchar(50)
    epoch_start_time_sec: float
    epoch_end_time_sec: float
    epoch_start_time_nspike: int
    epoch_end_time_nspike: int
    """
    
    def make(self, key):
        anim_name, day, day_path_raw = (Animal() * (Day() & key)).fetch1('anim_name', 'day', 'day_path_raw')
        try:
            times_mat = loadmat(os.path.join(day_path_raw, 'times.mat'))
            time_ranges = times_mat['ranges']
            names = times_mat['names']
            for epoch_id, epoch_time_range in enumerate(time_ranges[1:]):
                epoch_start_time_nspike = epoch_time_range[0]
                epoch_end_time_nspike = epoch_time_range[1]
                epoch_start_time_sec = epoch_start_time_nspike/10000
                epoch_end_time_sec = epoch_end_time_nspike/10000
                name_entry = names[epoch_id + 1][0][0]
                name_re = re.search('\d*\s*(\w*)\s*([0-9:\-_]*)$', name_entry)
                if name_re:
                    epoch_name = name_re.groups()[0]
                    epoch_time_str = name_re.groups()[1]
                    self.insert1({'anim_name': anim_name,
                                  'day': day, 
                                  'epoch_id': epoch_id, 
                                  'epoch_name': epoch_name,
                                  'epoch_time_str': epoch_time_str,
                                  'epoch_start_time_sec': epoch_start_time_sec,
                                  'epoch_end_time_sec':epoch_start_time_sec,
                                  'epoch_start_time_nspike': epoch_start_time_nspike,
                                  'epoch_end_time_nspike': epoch_end_time_nspike
                                 })

        except FileNotFoundError:
            raise CorruptData('Missing {:s}.'.format((os.path.join(day_path_raw, 'times.mat'))))
    
@schema
class Tetrode(dj.Imported):
    definition = """
    -> Animal
    elec_grp_id: tinyint
    ---
    tet_hemisphere: varchar(50)
    """
    def make(self, key):
        anim_name, day_path_mat = (Animal() & key).fetch1('anim_name', 'anim_path_mat')
        mat = loadmat(os.path.join(day_path_mat, 'bontetinfo.mat'))
        tet = {}
        for tet_days in mat['tetinfo'][0]:
            if len(tet_days[0]) > 0:
                for tet_epochs in tet_days[0]:
                    for tet_id, tet_epoch in enumerate(tet_epochs[0]):
                        if len(tet_epoch[0]) > 0:
                            tet_entry = tet.setdefault(tet_id+1, {})
                            tet_entry_hemi_list = tet_entry.setdefault('hemisphere', [])
                            try:
                                tet_hemi = tet_epoch[0][0]['hemisphere'][0]
                            except ValueError:
                                tet_hemi = None
                            tet_entry_hemi_list.append(tet_hemi)

        for tet_id, tet_entries in tet.items():
            tet_hemi = tet_entries['hemisphere']
            tet_hemi_set = set(tet_hemi)
            if len(tet_hemi_set) == 1:
                tet_hemisphere = list(tet_hemi_set)[0]
                if tet_hemisphere is None:
                    tet_hemisphere = ''
                self.insert1({'anim_name': anim_name,
                              'elec_grp_id': tet_id,
                              'tet_hemisphere': tet_hemisphere
                             })
            else:
                warnings.warn("Tetrode {:d} doesn't have exactly 1 type for hemisphere entry: {:s}".format(tet_id, str(tet_hemi_set)))

        
        
@schema
class TetrodeEpoch(dj.Manual):
    definition = """
    -> Tetrode
    -> Epoch
    ---
    tet_depth: int
    tet_num_cells: int
    tet_area: varchar(50)
    tet_subarea: varchar(50)
    """
    
@schema
class LFP(dj.Imported):
    definition = """
    -> TetrodeEpoch
    ---
    lfp_path_raw_part: varchar(200)
    lfp_path_raw_abs: varchar(200)
    lfp_path_eeg_part: varchar(200)
    lfp_path_eeg_abs: varchar(200)
    """
    
    def make(self, key):
        print (key)
        display((Animal() * Tetrode()) & key)
     
@schema
class RippleInterval(dj.Imported):
    definition = """
    -> Epoch
    algorithm: varchar(20)
    ---
    part_path: varchar(200)
    path_abs: varchar(200)
    """
    
    class LFPSource(dj.Part):
        definition = """
        -> LFP
        ---
        
        """
    
    def make(self, key):
        print(key)
                              

In [167]:
anim = Animal()
day = Day()
epoch = Epoch()
tet = Tetrode()
tet_ep = TetrodeEpoch()
lfp = LFP()
rip = RippleInterval()

In [168]:
anim.insert1({'anim_name': 'Bond', 'anim_path_raw': '/opt/data/daliu/other/mkarlsso/bond/', 'anim_path_mat': '/opt/data/daliu/other/mkarlsso/Bon/'})
display(anim)
day.populate()
display(day)
epoch.populate()
display(epoch)
tet.populate()
display(tet)